# Spark DataFrames Project Exercise

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.


#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

In [1]:
!apt-get -y install openjdk-8-jre-headless
!pip install pyspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jre-headless is already the newest version (8u312-b07-0ubuntu1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#### Start a simple Spark Session

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Walmart').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [3]:
df = spark.read.csv("walmart_stock.csv", header=True, inferSchema=True)

#### What are the column names?

In [4]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [5]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [6]:
for row in df.head(5):
    print(row)
    print("\n")

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




#### Use describe() to learn about the DataFrame.

In [7]:
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [8]:
# Uh oh Strings! 
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [9]:
from pyspark.sql.functions import format_number

In [10]:
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+
|summary|    Open|  Volume|
+-------+--------+--------+
|  count|1,258.00|    1258|
|   mean|   72.36| 8222093|
| stddev|    6.77| 4519780|
|    min|   56.39| 2094900|
|    max|   90.80|80898100|
+-------+--------+--------+



**Check Point 3**

format Open, High, Low, Close, Volume,  Adj Close

In [11]:
result = df.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [12]:
df2 = df.withColumn("HV Ratio", df["high"]/df["Volume"])
# df2.show()
df2.select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [13]:
# Didn't need to really do this much indexing
# Could have just shown the entire row
df.orderBy(df["High"].desc()).head(1)[0][0]

'2015-01-13'

#### What is the mean of the Close column?

In [14]:
# Also could have gotten this from describe()
from pyspark.sql.functions import mean
df.select(mean("close")).show()

+-----------------+
|       avg(close)|
+-----------------+
|72.38844998012726|
+-----------------+



**Check Point 4**

What is the max and min of the Volume column?

In [15]:
# Could have also used describe
from pyspark.sql.functions import min, max

In [16]:
df.select(max("Volume"), min("Volume")).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [17]:
df.filter("Close < 60").count()

81

In [18]:
df.filter(df["Close"] < 60).count()

81

In [19]:
from pyspark.sql.functions import count
result = df.filter(df["Close"] < 60)
result.select(count("Close")).show()

+------------+
|count(Close)|
+------------+
|          81|
+------------+



#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [20]:
(df.filter(df["High"] > 80).count() * 1.0 / df.count()) * 100

9.141494435612083

#### What is the Pearson correlation between High and Volume?


In [21]:
from pyspark.sql.functions import corr
df.select(corr("High", "Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [22]:
from pyspark.sql.functions import year
yeardf = df.withColumn("Year", year(df["Date"]))

In [23]:
max_df = yeardf.groupBy("Year").max()
# 2015
max_df.select("Year", "max(High)").show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+




#### What is the max Close for each Calendar Month?
#### In other words, across all the years, what is the max Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months.

In [24]:
from pyspark.sql.functions import month
monthdf = df.withColumn("Month", month("Date"))
monthmax = monthdf.select("Month", "Close").groupBy("Month").max()
monthmax.select("Month", "max(Close)").orderBy("Month").show()

+-----+-----------------+
|Month|       max(Close)|
+-----+-----------------+
|    1|        90.470001|
|    2|87.33000200000001|
|    3|        83.959999|
|    4|        81.029999|
|    5|79.91999799999999|
|    6|            77.32|
|    7|        78.550003|
|    8|        78.769997|
|    9|        77.510002|
|   10|        78.290001|
|   11|        87.540001|
|   12|        86.910004|
+-----+-----------------+



**Check Point 5**

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [25]:
monthdf = df.withColumn("Month", month("Date"))
monthmean = monthdf.select("Month", "Close").groupBy("Month").mean()
monthmean.select("Month", "avg(Close)").orderBy("Month").show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!
## Remember to upload this notebook file to new E3
## Check point 6: 0.5 points